In [1]:
import pickle

import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [2]:
from sklearn.pipeline import make_pipeline

In [3]:
import mlflow
mlflow.set_tracking_uri("sqlite://///home/frauke/ml-ops-zoomcamp/week04_Model_Deployment/batch/mlruns.db")
#mlflow.set_tracking_uri("sqlite:///mlruns.db")
mlflow.set_experiment("green-taxi-duration")

<Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='green-taxi-duration', tags={}>

In [4]:
def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds()
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df


def prepare_dictionaries(df: pd.DataFrame):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [5]:
df_train = read_dataframe('../../data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('../../data/green_tripdata_2021-02.parquet')

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_val)

In [6]:
with mlflow.start_run():
    params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=0)
    mlflow.log_params(params)

    pipeline = make_pipeline(
        DictVectorizer(),
        RandomForestRegressor(**params, n_jobs=-1)
    )

    dv = DictVectorizer()
    dv.fit(dict_train)
    pipeline.fit(dict_train, y_train)
    y_pred = pipeline.predict(dict_val)

    rmse = mean_squared_error(y_pred, y_val, squared=False)
    print(params, rmse)
    mlflow.log_metric('rmse', rmse)
    
    mlflow.sklearn.log_model(pipeline, artifact_path="model")

{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 15.136777093556063


In [7]:
from mlflow.tracking import MlflowClient

In [10]:
MLFLOW_TRACKING_URI = 'sqlite://///home/frauke/ml-ops-zoomcamp/week04_Model_Deployment/batch/mlruns.db'#'sqlite:///mlruns.db'
RUN_ID = 'e60b4352b7134f679cb04d95288a24c0'

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [11]:
path = client.download_artifacts(run_id=RUN_ID, path='model/model.pkl')

In [12]:
with open(path, 'rb') as f_out:
    pipline = pickle.load(f_out)

In [13]:
pipeline[0]

DictVectorizer()